In [1]:
import pandas as pd
import subprocess
import os
import json
import exifread

In [2]:
# set directory
directory = "photos_original"
photos = []
photos_names = []

In [3]:
# iterate over files in that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        with open (f, "rb") as photo:
            print(f.title())
            photos_names.append(f.title())
            tags = exifread.process_file(photo)
            photos.append(tags)

File format not recognized.
Possibly corrupted field Tag 0x001A in MakerNote IFD
Possibly corrupted field Tag 0x001A in MakerNote IFD
Possibly corrupted field Tag 0x001A in MakerNote IFD


photos_original/IMG_2575.png
Photos_Original/Img_2575.Png
photos_original/IMG_5226.png
Photos_Original/Img_5226.Png
photos_original/IMG_1280.png
Photos_Original/Img_1280.Png
photos_original/IMG_2330.mov
Photos_Original/Img_2330.Mov
photos_original/IMG_5345.png
Photos_Original/Img_5345.Png
photos_original/IMG_2563.png
Photos_Original/Img_2563.Png
photos_original/IMG_4516.png
Photos_Original/Img_4516.Png
photos_original/IMG_1644.png
Photos_Original/Img_1644.Png
photos_original/IMG_5126.png
Photos_Original/Img_5126.Png
photos_original/IMG_2918.png
Photos_Original/Img_2918.Png
photos_original/IMG_1419.png
Photos_Original/Img_1419.Png
photos_original/IMG_1420.png
Photos_Original/Img_1420.Png
photos_original/IMG_4599.png
Photos_Original/Img_4599.Png
photos_original/IMG_2506.png
Photos_Original/Img_2506.Png
photos_original/IMG_2331.png
Photos_Original/Img_2331.Png
photos_original/IMG_4595.png
Photos_Original/Img_4595.Png
photos_original/IMG_0902.png
Photos_Original/Img_0902.Png
photos_origina

In [4]:
photos_df = pd.DataFrame(photos)
photos_df['title'] = photos_names
photos_df

,Image Make,Image Model,Image Orientation,Image XResolution,Image YResolution,Image ResolutionUnit,Image Software,Image DateTime,Image TileWidth,Image TileLength,...,MakerNote Tag 0x000F,MakerNote Tag 0x001C,EXIF UserComment,MakerNote Tag 0x0011,MakerNote Tag 0x0028,MakerNote Tag 0x0012,EXIF CustomRendered,EXIF DigitalZoomRatio,MakerNote HDRImageType,title
0,Apple,iPhone 8 Plus,Horizontal (normal),72,72,Pixels/Inch,12.1.4,2019:03:14 13:54:11,512,512,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_2575.Png
1,Apple,iPhone 6s Plus,Horizontal (normal),72,72,Pixels/Inch,11.4,2018:06:12 21:48:39,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_5226.Png
2,Apple,iPhone 8 Plus,Horizontal (normal),72,72,Pixels/Inch,12.0.1,2018:11:14 13:55:19,512,512,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_1280.Png
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_2330.Mov
4,Apple,iPhone 6s Plus,Horizontal (normal),72,72,Pixels/Inch,11.4,2018:07:02 12:25:19,NaN,NaN,...,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_5345.Png
5,Apple,iPhone 6s Plus,Horizontal (normal),72,72,Pixels/Inch,10.3.2,2017:08:16 15:25:32,NaN,NaN,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_2563.Png
6,Apple,iPhone 6s Plus,Horizontal (normal),72,72,Pixels/Inch,11.2.6,2018:03:30 10:54:32,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_4516.Png
7,NaN,NaN,Horizontal (normal),NaN,NaN,NaN,NaN,2018:12:13 13:05:55,NaN,NaN,...,NaN,NaN,Screenshot,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_1644.Png
8,Apple,iPhone 6s Plus,Horizontal (normal),72,72,Pixels/Inch,11.2.6,2018:05:31 08:46:04,NaN,NaN,...,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_5126.Png
9,Apple,iPhone 6s Plus,Horizontal (normal),72,72,Pixels/Inch,10.3.3,2017:09:11 10:32:14,NaN,NaN,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_2918.Png


In [5]:
photos_df.columns

Index(['Image Make', 'Image Model', 'Image Orientation', 'Image XResolution',
       'Image YResolution', 'Image ResolutionUnit', 'Image Software',
       'Image DateTime', 'Image TileWidth', 'Image TileLength',
       'Image ExifOffset', 'GPS GPSLatitudeRef', 'GPS GPSLatitude',
       'GPS GPSLongitudeRef', 'GPS GPSLongitude', 'GPS GPSAltitudeRef',
       'GPS GPSAltitude', 'GPS GPSSpeedRef', 'GPS GPSSpeed',
       'GPS GPSImgDirectionRef', 'GPS GPSImgDirection',
       'GPS GPSDestBearingRef', 'GPS GPSDestBearing', 'GPS GPSDate',
       'GPS Tag 0x001F', 'Image GPSInfo', 'EXIF ExposureTime', 'EXIF FNumber',
       'EXIF ExposureProgram', 'EXIF ISOSpeedRatings', 'EXIF ExifVersion',
       'EXIF DateTimeOriginal', 'EXIF DateTimeDigitized', 'EXIF OffsetTime',
       'EXIF OffsetTimeOriginal', 'EXIF OffsetTimeDigitized',
       'EXIF ComponentsConfiguration', 'EXIF ShutterSpeedValue',
       'EXIF ApertureValue', 'EXIF BrightnessValue', 'EXIF ExposureBiasValue',
       'EXIF MeteringMode

In [6]:
photos_df['latitude'] = photos_df['GPS GPSLatitude']
photos_df['longitude'] = photos_df['GPS GPSLongitude']
photos_df['datetime'] = photos_df['EXIF DateTimeOriginal']

photos_df

,Image Make,Image Model,Image Orientation,Image XResolution,Image YResolution,Image ResolutionUnit,Image Software,Image DateTime,Image TileWidth,Image TileLength,...,MakerNote Tag 0x0011,MakerNote Tag 0x0028,MakerNote Tag 0x0012,EXIF CustomRendered,EXIF DigitalZoomRatio,MakerNote HDRImageType,title,latitude,longitude,datetime
0,Apple,iPhone 8 Plus,Horizontal (normal),72,72,Pixels/Inch,12.1.4,2019:03:14 13:54:11,512,512,...,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_2575.Png,"[40, 12, 788/25]","[74, 2, 583/25]",2019:03:14 13:54:11
1,Apple,iPhone 6s Plus,Horizontal (normal),72,72,Pixels/Inch,11.4,2018:06:12 21:48:39,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_5226.Png,"[40, 4, 3713/100]","[74, 7, 1342/25]",2018:06:12 21:48:39
2,Apple,iPhone 8 Plus,Horizontal (normal),72,72,Pixels/Inch,12.0.1,2018:11:14 13:55:19,512,512,...,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_1280.Png,"[40, 12, 1571/50]","[74, 2, 2507/100]",2018:11:14 13:55:19
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_2330.Mov,NaN,NaN,NaN
4,Apple,iPhone 6s Plus,Horizontal (normal),72,72,Pixels/Inch,11.4,2018:07:02 12:25:19,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_5345.Png,"[40, 4, 3831/100]","[74, 7, 5151/100]",2018:07:02 12:25:19
5,Apple,iPhone 6s Plus,Horizontal (normal),72,72,Pixels/Inch,10.3.2,2017:08:16 15:25:32,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_2563.Png,"[40, 4, 179/5]","[74, 7, 2563/50]",2017:08:16 15:25:32
6,Apple,iPhone 6s Plus,Horizontal (normal),72,72,Pixels/Inch,11.2.6,2018:03:30 10:54:32,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_4516.Png,"[40, 12, 156/5]","[74, 2, 2431/100]",2018:03:30 10:54:32
7,NaN,NaN,Horizontal (normal),NaN,NaN,NaN,NaN,2018:12:13 13:05:55,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_1644.Png,NaN,NaN,2018:12:13 13:05:55
8,Apple,iPhone 6s Plus,Horizontal (normal),72,72,Pixels/Inch,11.2.6,2018:05:31 08:46:04,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_5126.Png,"[40, 12, 811/25]","[74, 2, 149/5]",2018:05:31 08:46:04
9,Apple,iPhone 6s Plus,Horizontal (normal),72,72,Pixels/Inch,10.3.3,2017:09:11 10:32:14,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Photos_Original/Img_2918.Png,"[40, 4, 3731/100]","[74, 7, 1274/25]",2017:09:11 10:32:14


In [7]:
# photos_df.dropna(subset=['datetime'], inplace=True)
photos_df['datetime']

0     2019:03:14 13:54:11
1     2018:06:12 21:48:39
2     2018:11:14 13:55:19
3                     NaN
4     2018:07:02 12:25:19
5     2017:08:16 15:25:32
6     2018:03:30 10:54:32
7     2018:12:13 13:05:55
8     2018:05:31 08:46:04
9     2017:09:11 10:32:14
10    2018:11:22 16:42:03
11    2018:11:22 16:42:06
12    2018:08:04 14:54:40
13    2017:08:11 18:31:06
14    2019:02:11 20:13:54
15    2018:08:04 14:54:32
16    2018:11:22 16:08:30
17    2018:06:27 11:52:17
18    2019:01:18 15:17:03
19    2018:09:26 11:21:49
20    2018:08:04 14:54:37
21    2019:01:21 10:02:03
22    2019:01:13 21:45:09
23    2019:01:21 10:02:01
24    2017:09:11 10:32:33
25    2018:11:29 17:02:45
26    2018:08:07 18:59:00
27    2018:11:16 10:48:00
28    2018:11:16 10:47:37
29    2019:04:06 11:20:01
30    2018:11:16 10:47:39
31    2018:11:30 12:12:38
32    2018:08:31 13:16:06
33    2018:08:03 09:21:52
34    2017:08:17 09:51:12
35    2017:06:28 21:06:44
36    2017:08:17 09:27:01
Name: datetime, dtype: object

In [8]:
photos_important_info_cols = ['title', 'datetime', 'latitude', 'longitude']
photos_important_info_df = pd.DataFrame(photos_df, columns=photos_important_info_cols)

In [9]:
photos_important_info_df

,title,datetime,latitude,longitude
0,Photos_Original/Img_2575.Png,2019:03:14 13:54:11,"[40, 12, 788/25]","[74, 2, 583/25]"
1,Photos_Original/Img_5226.Png,2018:06:12 21:48:39,"[40, 4, 3713/100]","[74, 7, 1342/25]"
2,Photos_Original/Img_1280.Png,2018:11:14 13:55:19,"[40, 12, 1571/50]","[74, 2, 2507/100]"
3,Photos_Original/Img_2330.Mov,NaN,NaN,NaN
4,Photos_Original/Img_5345.Png,2018:07:02 12:25:19,"[40, 4, 3831/100]","[74, 7, 5151/100]"
5,Photos_Original/Img_2563.Png,2017:08:16 15:25:32,"[40, 4, 179/5]","[74, 7, 2563/50]"
6,Photos_Original/Img_4516.Png,2018:03:30 10:54:32,"[40, 12, 156/5]","[74, 2, 2431/100]"
7,Photos_Original/Img_1644.Png,2018:12:13 13:05:55,NaN,NaN
8,Photos_Original/Img_5126.Png,2018:05:31 08:46:04,"[40, 12, 811/25]","[74, 2, 149/5]"
9,Photos_Original/Img_2918.Png,2017:09:11 10:32:14,"[40, 4, 3731/100]","[74, 7, 1274/25]"


In [10]:
photos_important_info_df.to_csv("photos_info.csv")